# Cordic实验
- 完成功能：1.输入横坐标x与纵坐标y，可得坐标对应的模长及角度    2.输入角度（0-90），可得角度对应的正弦与余弦值
- 说明：1.vivado工程共导入两个ip核，分别完成功能1与功能2
     
## 1.导入bit文件

In [63]:
import math
from pynq import Overlay
overlay = Overlay("../overlay/cordic_new.bit")
cordic = overlay.cordiccart2pol_0  #实现功能1
cordic_2=overlay.cordic_0    #实现功能2

## 2.查看寄存器 

In [64]:
cordic.register_map #x与y代表横纵坐标，r代表模长，theta代表角度

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=write-only),
  y = Register(y=write-only),
  r = Register(r=0),
  r_ctrl = Register(r_ap_vld=0, RESERVED=0),
  theta = Register(theta=0),
  theta_ctrl = Register(theta_ap_vld=0, RESERVED=0)
}

In [65]:
cordic_2.register_map #theta代表输入角度，s代表正弦值，c代表余弦值

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  theta = Register(theta=write-only, RESERVED=write-only),
  s = Register(s=0, RESERVED=0),
  s_ctrl = Register(s_ap_vld=0, RESERVED=0),
  c = Register(c=0, RESERVED=0),
  c_ctrl = Register(c_ap_vld=0, RESERVED=0)
}

### 3.1功能1的实现（输入横坐标x与纵坐标y，可得坐标对应的模长及角度）

#### 3.1.1输入横纵坐标的x与y

In [66]:
import struct
#########################input############################
x_input=1;
y_input=1.732;

def floatToBytes(f):
    bs = struct.pack("f",f)
    return (bs[3]<<24)+(bs[2]<<16)+(bs[1]<<8)+bs[0]

x=floatToBytes(x_input)  #将输入浮点数转换为二进制
y=floatToBytes(y_input)  
cordic.write(0x10,x)     #写入相应的寄存器中
cordic.write(0x18,y)

#### 3.1.2开始执行

In [67]:
import time

cordic.register_map.CTRL.AP_START = 1

start_time = time.time()

while True:
    reg = cordic.register_map.CTRL.AP_START
    if reg != 1:
        break

end_time = time.time()

print("耗时：{}s".format(end_time - start_time))

耗时：0.0009248256683349609s


#### 3.1.3 读出坐标的模值

In [68]:
r=cordic.read(0x20)

def bytesToFloat(h1,h2,h3,h4):
    ba = bytearray()
    ba.append(h1)
    ba.append(h2)
    ba.append(h3)
    ba.append(h4)
    return struct.unpack("!f",ba)[0]

h1=(r&0xff000000)>>24
h2=(r&0x00ff0000)>>16
h3=(r&0x0000ff00)>>8
h4=(r&0x000000ff)
print("模值为")
print(bytesToFloat(h1,h2,h3,h4))  ##将二进制转换为浮点数

模值为
1.9999561309814453


#### 3.1.4 读出坐标的角度

In [69]:
theta=cordic.read(0x30)
h1=(theta&0xff000000)>>24
h2=(theta&0x00ff0000)>>16
h3=(theta&0x0000ff00)>>8
h4=(theta&0x000000ff)
print("角度为")
print(bytesToFloat(h1,h2,h3,h4)/math.pi*180)

角度为
59.9991615563842


### 3.2功能2的实现（输入角度，输出对应的sin与cos）

#### 3.2.1输入角度degree

In [70]:
degree=60          #输入0-90之间的角度
radian=degree/180*math.pi

#------------------将浮点数radian转换为定点数
FL = 10  # 小数部分位数
IL = 12-FL  # 整数部分位数
round = False  # False 是最近邻取舍，True 是随机取舍
MIN = -(1 << (IL - 1))
MAX = -MIN - 2 ** (-FL)
# 量化范围是[  -2^(IL-1)~2^(IL-1)-2^(-IF)  ]
def float_to_fixed(x):

    global MIN, MAX, FL
    #print(FL)
    if x <= MIN: return MIN
    if x >= MAX: return MAX
    sig = 1
    if x < 0:
        sig = -1
        x = -x
    q = int(x)
    x -= q
    e = 1
    for i in range(FL):
        x *= 2
        e /= 2
        if x >= 1:
            x -= 1
            q += e

    if round:  # 随机舍入
        r = random()  # 产生0-1的随机数
        if r < x:
            q += e
    else:  # 邻近舍入
        if x >= 0.5:
            q += e
    q *= sig
    if q <= MIN: return MIN
    if q >= MAX: return MAX
    return q
#-----------------------------------

q=float_to_fixed(radian)  ##将浮点数转换为定点数
q_qunt=int(q*2**10)      ##量化
#print(q_qunt)

cordic_2.write(0x10,q_qunt)     #写入相应的寄存器中

#### 3.2.2开始执行

In [71]:
import time

cordic_2.register_map.CTRL.AP_START = 1

start_time = time.time()

while True:
    reg = cordic_2.register_map.CTRL.AP_START
    if reg != 1:
        break

end_time = time.time()

print("耗时：{}s".format(end_time - start_time))

耗时：0.0008766651153564453s


#### 3.2.3读出sin与cos值

In [72]:
sinx=cordic_2.read(0x18)
print("正弦值为")
print(sinx/2**10)

正弦值为
0.865234375


In [73]:
cosx=cordic_2.read(0x28)
print("余弦值为")
print(cosx/2**10)

余弦值为
0.498046875
